In [46]:
!pip install bokeh


In [47]:
!pip install pathway
import pathway as pw
import pandas as pd

# Simulated CSV loading (you can replace with actual streaming later)
df = pd.read_csv("dataset.csv")
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format="%d-%m-%Y %H:%M:%S"
)

df = df.sort_values(by=["SystemCodeNumber", "Timestamp"])

# Export as a fresh CSV to simulate streaming
df.to_csv("stream.csv", index=False)


In [48]:
class ParkingLotSchema(pw.Schema):
    ID: str
    SystemCodeNumber: str
    Capacity: int
    Occupancy: int
    QueueLength: int
    VehicleType: str
    TrafficConditionNearby: str
    IsSpecialDay: int
    LastUpdatedDate: str
    LastUpdatedTime: str


In [49]:
@pw.udf
def combine_timestamp(date_str: str, time_str: str) -> str:
    # Combine and parse with the correct format
    dt = pd.to_datetime(date_str + ' ' + time_str, format="%d-%m-%Y %H:%M:%S")
    return dt.isoformat()  # ISO format string for Bokeh compatibility


In [50]:
table = pw.io.csv.read(
    "stream.csv",
    schema=ParkingLotSchema,
    mode="static",
    autocommit_duration_ms=500,
)


In [51]:
table += table.select(
    Timestamp=combine_timestamp(table.LastUpdatedDate, table.LastUpdatedTime)
)



In [52]:
# # table = pw.io.csv.read(
# #     "stream.csv",
# #     schema=ParkingLotSchema,
# #     mode="static",  # or streaming if deployed
# #     autocommit_duration_ms=500,
# # )

# # Add computed timestamp
# table += table.select(Timestamp=table.Timestamp)



In [53]:
from sklearn.preprocessing import MinMaxScaler

@pw.udf
def calculate_price(capacity, occupancy, queue, traffic, special_day, vehicle_type):
    # Encode features
    traffic_map = {'low': 0, 'medium': 1, 'high': 2}
    vehicle_map = {'bike': 0.5, 'car': 1.0, 'truck': 1.5}

    traffic_num = traffic_map.get(traffic, 1)
    vehicle_weight = vehicle_map.get(vehicle_type, 1.0)
    occupancy_rate = occupancy / capacity if capacity else 0

    # Demand Function
    raw_demand = (
        2.0 * occupancy_rate +
        0.5 * queue -
        1.0 * traffic_num +
        1.5 * special_day +
        1.0 * vehicle_weight
    )

    # Normalize demand (simple linear scaling here; can also cache past min/max per lot)
    norm_demand = max(0, min(1, (raw_demand - 1) / 10))  # crude normalization

    # Price calculation
    base_price = 10
    price = base_price * (1 + 1.0 * norm_demand)
    return round(min(max(price, 5), 20), 2)


In [54]:
# Compute price
table += table.select(
    calculatedPrice=calculate_price(
        table.Capacity,
        table.Occupancy,
        table.QueueLength,
        table.TrafficConditionNearby,
        table.IsSpecialDay,
        table.VehicleType
    )
)

# # Extract for visualization
output = table.select(
    LotID=table.SystemCodeNumber,
    Timestamp=table.Timestamp,
     finalPrice=table.calculatedPrice
)
pw.io.csv.write(
    table=output,
    filename="output.csv",
    # with_header=True
)

pw.run()



Output()

In [56]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import column

# ---- STEP 8: Convert to Pandas and Plot ----
# Use pw.outputs() to get the output table and then convert it to a pandas DataFrame
df_out = pd.read_csv("output.csv")
df_out["Timestamp"] = pd.to_datetime(df_out["Timestamp"])

# ---- STEP 9: Bokeh Visualization ----
plots = []
for lot_id, group in df_out.groupby("LotID"):
    group = group.sort_values("Timestamp")
    source = ColumnDataSource(group)

    p = figure(
        title=f"Price Over Time - Lot {lot_id}",
        x_axis_type="datetime", width=800, height=300
    )
    p.line(x='Timestamp', y='finalPrice', source=source, line_width=2)
    p.circle(x='Timestamp', y='finalPrice', source=source, size=4, color="navy")
    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Price ($)"
    plots.append(p)

# Display all plots
show(column(*plots))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Reasoning**:
Import necessary libraries for Bokeh visualization and create the Bokeh plot for price over time.

